In [1]:
import torch

In [2]:
shape = (2,3,4,2) # B*L*E*N
range_num = 1
for i in shape:
    range_num *= i
s = torch.Tensor(range(range_num)).reshape(shape)
t = torch.Tensor([ list(reversed(list(a3))) for a1 in s for a2 in a1 for a3 in a2 ])
t = t.reshape(shape)

# conv 

In [3]:
hs = s.reshape((1,s.shape[0]*s.shape[1]*s.shape[2],s.shape[3]))
ht = t.reshape((t.shape[0]*t.shape[1]*t.shape[2],1,t.shape[3]))
h = torch.nn.functional.conv1d(hs, ht, groups = ht.shape[0], padding=1)

In [4]:
hs

tensor([[[ 0.,  1.],
         [ 2.,  3.],
         [ 4.,  5.],
         [ 6.,  7.],
         [ 8.,  9.],
         [10., 11.],
         [12., 13.],
         [14., 15.],
         [16., 17.],
         [18., 19.],
         [20., 21.],
         [22., 23.],
         [24., 25.],
         [26., 27.],
         [28., 29.],
         [30., 31.],
         [32., 33.],
         [34., 35.],
         [36., 37.],
         [38., 39.],
         [40., 41.],
         [42., 43.],
         [44., 45.],
         [46., 47.]]])

In [5]:
ht

tensor([[[ 1.,  0.]],

        [[ 3.,  2.]],

        [[ 5.,  4.]],

        [[ 7.,  6.]],

        [[ 9.,  8.]],

        [[11., 10.]],

        [[13., 12.]],

        [[15., 14.]],

        [[17., 16.]],

        [[19., 18.]],

        [[21., 20.]],

        [[23., 22.]],

        [[25., 24.]],

        [[27., 26.]],

        [[29., 28.]],

        [[31., 30.]],

        [[33., 32.]],

        [[35., 34.]],

        [[37., 36.]],

        [[39., 38.]],

        [[41., 40.]],

        [[43., 42.]],

        [[45., 44.]],

        [[47., 46.]]])

In [6]:
hs.shape

torch.Size([1, 24, 2])

In [7]:
h.shape

torch.Size([1, 24, 3])

In [8]:
h = h.squeeze(0)

In [9]:
s.shape[0],s.shape[1],s.shape[2],h.shape[1]

(2, 3, 4, 3)

In [10]:
h = h.reshape(s.shape[0],s.shape[1],s.shape[2],h.shape[1]).permute(0,1,3,2)
h.shape #B*L*(Ns+Nt-1)*E

torch.Size([2, 3, 3, 4])

In [11]:
h

tensor([[[[0.0000e+00, 4.0000e+00, 1.6000e+01, 3.6000e+01],
          [0.0000e+00, 1.2000e+01, 4.0000e+01, 8.4000e+01],
          [1.0000e+00, 9.0000e+00, 2.5000e+01, 4.9000e+01]],

         [[6.4000e+01, 1.0000e+02, 1.4400e+02, 1.9600e+02],
          [1.4400e+02, 2.2000e+02, 3.1200e+02, 4.2000e+02],
          [8.1000e+01, 1.2100e+02, 1.6900e+02, 2.2500e+02]],

         [[2.5600e+02, 3.2400e+02, 4.0000e+02, 4.8400e+02],
          [5.4400e+02, 6.8400e+02, 8.4000e+02, 1.0120e+03],
          [2.8900e+02, 3.6100e+02, 4.4100e+02, 5.2900e+02]]],


        [[[5.7600e+02, 6.7600e+02, 7.8400e+02, 9.0000e+02],
          [1.2000e+03, 1.4040e+03, 1.6240e+03, 1.8600e+03],
          [6.2500e+02, 7.2900e+02, 8.4100e+02, 9.6100e+02]],

         [[1.0240e+03, 1.1560e+03, 1.2960e+03, 1.4440e+03],
          [2.1120e+03, 2.3800e+03, 2.6640e+03, 2.9640e+03],
          [1.0890e+03, 1.2250e+03, 1.3690e+03, 1.5210e+03]],

         [[1.6000e+03, 1.7640e+03, 1.9360e+03, 2.1160e+03],
          [3.2800e+03, 3.612

# fft

In [12]:
from torch.fft import fft, ifft

In [13]:
length = s.shape[-1] + t.shape[-1] -1
s_ = fft(s, n=length)
t_ = fft(t, n=length)
h = s_*t_
h = ifft(h)
h = h.permute(0,1,3,2).float() #B*L*(Is+It-1)*E

<ipython-input-13-3d1f4345e411>:6: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /opt/conda/conda-bld/pytorch_1614378083779/work/aten/src/ATen/native/Copy.cpp:219.)
  h = h.permute(0,1,3,2).float() #B*L*(Is+It-1)*E


In [14]:
length

3

In [15]:
h

tensor([[[[0.0000e+00, 6.0000e+00, 2.0000e+01, 4.2000e+01],
          [1.0000e+00, 1.3000e+01, 4.1000e+01, 8.5000e+01],
          [1.7948e-08, 6.0000e+00, 2.0000e+01, 4.2000e+01]],

         [[7.2000e+01, 1.1000e+02, 1.5600e+02, 2.1000e+02],
          [1.4500e+02, 2.2100e+02, 3.1300e+02, 4.2100e+02],
          [7.2000e+01, 1.1000e+02, 1.5600e+02, 2.1000e+02]],

         [[2.7200e+02, 3.4200e+02, 4.2000e+02, 5.0600e+02],
          [5.4500e+02, 6.8500e+02, 8.4100e+02, 1.0130e+03],
          [2.7200e+02, 3.4200e+02, 4.2000e+02, 5.0600e+02]]],


        [[[6.0000e+02, 7.0200e+02, 8.1200e+02, 9.3000e+02],
          [1.2010e+03, 1.4050e+03, 1.6250e+03, 1.8610e+03],
          [6.0000e+02, 7.0200e+02, 8.1200e+02, 9.3000e+02]],

         [[1.0560e+03, 1.1900e+03, 1.3320e+03, 1.4820e+03],
          [2.1130e+03, 2.3810e+03, 2.6650e+03, 2.9650e+03],
          [1.0560e+03, 1.1900e+03, 1.3320e+03, 1.4820e+03]],

         [[1.6400e+03, 1.8060e+03, 1.9800e+03, 2.1620e+03],
          [3.2810e+03, 3.613